In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import json
import numpy as np
import imgaug as ia
from PIL import Image 
# import ImageEnhance
import ImageFilter
# from scipy.misc import imsave



In [2]:
from imgaug import augmenters as iaa

In [3]:
os.mkdir(maskt)

NameError: name 'maskt' is not defined

In [3]:
def plot_pair(images, gray=False):

    fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(10,8))
    i=0
    
    for y in range(2):
        if gray:
            axes[y].imshow(images[i], cmap='gray')
        else:
            axes[y].imshow(images[i])
        axes[y].get_xaxis().set_visible(False)
        axes[y].get_yaxis().set_visible(False)
        i+=1
    
    plt.show()

In [4]:
def get_poly(ann_path):
    
    with open(ann_path) as handle:
        data = json.load(handle)
    
    shape_dicts = data['shapes']
    
    return shape_dicts

In [5]:
def create_binary_masks(im, shape_dicts):
    
    blank = np.zeros(shape=(im.shape[0], im.shape[1]), dtype=np.float32)
    
    for shape in shape_dicts:
        points = np.array(shape['points'], dtype=np.int32)
        cv2.fillPoly(blank, [points], 255)
        
    return blank

In [13]:
path = '"C:\Users\DELL\Desktop\New folder (18)\Minor_dataset\Final_dataset\mask_im"'

In [ ]:
import os
import cv2
import json
import numpy as np 

source_folder = os.path.join(os.getcwd(), "images")
json_path = "maskGen_json.json"                     # Relative to root directory
count = 0                                           # Count of total images saved
file_bbs = {}                                       # Dictionary containing polygon coordinates for mask
MASK_WIDTH = 256				    # Dimensions should match those of ground truth image
MASK_HEIGHT = 256									

# Read JSON file
with open(json_path) as f:
  data = json.load(f)

# Extract X and Y coordinates if available and update dictionary
def add_to_dict(data, itr, key, count):
    try:
	x_points = data[itr]["regions"][count]["shape_attributes"]["all_points_x"]
	y_points = data[itr]["regions"][count]["shape_attributes"]["all_points_y"]
    except:
        print("No BB. Skipping", key)
        return
    
    all_points = []
    for i, x in enumerate(x_points):
        all_points.append([x, y_points[i]])
    
    file_bbs[key] = all_points
  
for itr in data:
    file_name_json = data[itr]["filename"]
    sub_count = 0               # Contains count of masks for a single ground truth image
    
    if len(data[itr]["regions"]) > 1:
        for _ in range(len(data[itr]["regions"])):
            key = file_name_json[:-4] + "*" + str(sub_count+1)
            add_to_dict(data, itr, key, sub_count)
            sub_count += 1
    else:
        add_to_dict(data, itr, file_name_json[:-4], 0)

			
print("\nDict size: ", len(file_bbs))

for file_name in os.listdir(source_folder):
    to_save_folder = os.path.join(source_folder, file_name[:-4])
    image_folder = os.path.join(to_save_folder, "images")
    mask_folder = os.path.join(to_save_folder, "masks")
    curr_img = os.path.join(source_folder, file_name)
    
    # make folders and copy image to new location
    os.mkdir(to_save_folder)
    os.mkdir(image_folder)
    os.mkdir(mask_folder)
    os.rename(curr_img, os.path.join(image_folder, file_name))
        
# For each entry in dictionary, generate mask and save in correponding 
# folder
for itr in file_bbs:
    num_masks = itr.split("*")
    to_save_folder = os.path.join(source_folder, num_masks[0])
    mask_folder = os.path.join(to_save_folder, "masks")
    mask = np.zeros((MASK_WIDTH, MASK_HEIGHT))
    try:
        arr = np.array(file_bbs[itr])
    except:
        print("Not found:", itr)
        continue
    count += 1
    cv2.fillPoly(mask, [arr], color=(255))
    
    if len(num_masks) > 1:
    	cv2.imwrite(os.path.join(mask_folder, itr.replace("*", "_") + ".png") , mask)    
    else:
        cv2.imwrite(os.path.join(mask_folder, itr + ".png") , mask)
        
print("Images saved:", count)

In [11]:
source_folder = os.path.join(os.getcwd(), "imagess")

In [12]:
for file_name in os.listdir(source_folder):
    to_save_folder = os.path.join(source_folder, file_name[:-4])
    image_folder = os.path.join(to_save_folder, "im")
    mask_folder = os.path.join(to_save_folder, "masks")
    curr_img = os.path.join(source_folder, file_name)
    
    # make folders and copy image to new location
    os.mkdir(to_save_folder)
    os.mkdir(image_folder)
    os.mkdir(mask_folder)
    os.rename(curr_img, os.path.join(image_folder, file_name))

In [26]:
image_list = sorted(os.listdir('images_jpg'), key=lambda x: int(x.split('.')[0]))
annot_list = sorted(os.listdir('mask_jpg'), key=lambda x: int(x.split('.')[0]))


In [8]:
import os
filelist=os.listdir('images_jpg')
for fichier in filelist[:]: # filelist[:] makes a copy of filelist.
    if not(fichier.endswith("jpeg")):
        filelist.remove(fichier)
print(filelist)

['63.jpeg', '64.jpeg']


### Create Masks for Binary Classification

In [1]:
image_list = sorted(os.listdir('image_256'), key=lambda x: int(x.split('.')[0]))
annot_list = sorted(os.listdir('mask_256'), key=lambda x: int(x.split('.')[0]))

for im_fn ,ann_fn in zip(image_list,annot_list):
    
    im = cv2.imread(os.path.join('image_256', im_fn))
    
    ann_path = os.path.join('mask_256', ann_fn)
    shape_dicts = get_poly(ann_path)
    im_binary = create_binary_masks(im, shape_dicts)
    
    
    cv2.imwrite(os.path.join(path , im_fn), im_binary)
    
    
        
    
    
#     cv2.fillPoly(blank, [label2poly[label]], (hues[label], 255, 255))
    
    
    plot_pair([im, im_binary], gray=True)
    plt.show()
#     print(im_fn,ann_fn)
    

NameError: name 'os' is not defined

### Create Masks for Multi Classification

In [6]:
hues = {'Onion1': 0,
        'Onion2': 0,
        'Onion3': 0,
        'Weed11': 60,
        'Weed12': 60,
        'Weed13': 60,
        'Weed21': 60,
        'Weed22': 60,
        'Weed23': 60,
        'Weed24': 60,
        'Weed31': 60,
        'Weed32': 60,
        'Weed33': 60,
        'Weed34': 60,
        
        }
# 
labels = sorted(hues.keys())
print(labels)

['Onion1', 'Onion2', 'Onion3', 'Weed11', 'Weed12', 'Weed13', 'Weed21', 'Weed22', 'Weed23', 'Weed24', 'Weed31', 'Weed32', 'Weed33', 'Weed34']


To set the hues you want, refer to this pi chart. These are the values used by OpenCV.

![hsv](docs/images/hsv.png)

In [7]:
def create_multi_masks(im, shape_dicts):
    
    channels = []
    cls = [x['label'] for x in shape_dicts]
    poly = [np.array(x['points'], dtype=np.int32) for x in shape_dicts]
    label2poly = dict(zip(cls, poly))
    background = np.zeros(shape=im.shape, dtype=np.float32)

    for i, label in enumerate(labels):
        
        blank = np.zeros(shape=im.shape, dtype=np.float32)
        
        if label in cls:
            cv2.fillPoly(blank, [label2poly[label]], 255)
            cv2.fillPoly(background, [label2poly[label]], 255)
            
        channels.append(blank)
    _, thresh = cv2.threshold(background, 127, 255, cv2.THRESH_BINARY_INV)
    channels.append(thresh)

    Y = np.stack(channels, axis=2)
    return Y

In [2]:
for im_fn, ann_fn in zip(image_list, annot_list):
    
    im = cv2.imread(os.path.join('images', im_fn), 0)
    ann_path = os.path.join('mask', ann_fn)
    shape_dicts = get_poly(ann_path)
    Y = create_multi_masks(im, shape_dicts)
    break
    
for i in range(len(im)):
    mask = Y[:,:,i]
    plt.imshow(mask, cmap='gray')
    plt.show()

NameError: name 'image_list' is not defined

### Hue Masks

In [8]:
def draw_multi_masks(im, shape_dicts):
    
    blank = np.zeros(shape=im.shape, dtype=np.uint8)
    
    channels = []
    cls = [x['label'] for x in shape_dicts]
    poly = [np.array(x['points'], dtype=np.int32) for x in shape_dicts]
    label2poly = dict(zip(cls, poly))

    for i, label in enumerate(labels):
        
        if label in cls:
            cv2.fillPoly(blank, [label2poly[label]], (hues[label], 255, 255))
            
    return cv2.cvtColor(blank, cv2.COLOR_HSV2RGB)

In [3]:
for i, (im_fn, ann_fn) in enumerate(zip(image_list, annot_list)):
    
    im = cv2.imread(os.path.join('"C:\Users\DELL\Desktop\New folder (18)\Minor_dataset\Final_dataset\imagesss"', im_fn), 1)
    ann_path = os.path.join('mask_jpg', ann_fn)
    shape_dicts = get_poly(ann_path)
    im_color = draw_multi_masks(im, shape_dicts)
#     thresh = 128
#     img_binary = cv2.threshold(im_color, thresh, 255, cv2.THRESH_BINARY)[1]
    
#     image_file = Image.open("convert_image.png") # open colour image
#     im_binary = im_color.convert('1') # convert image to black and white
#     image_file.save('result.png')
    cv2.imwrite(os.path.join(path , im_fn), im_color)
    display(Image.fromarray(im_color))
#     if i == 4:
#         break

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 3-4: truncated \UXXXXXXXX escape (<ipython-input-3-de9b33499265>, line 3)

In [19]:
def augment_poly(im_path, ann_path):
    
    im = cv2.imread(im_path, 1)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    
    with open(ann_path) as handle:
        data = json.load(handle)
    
    shape_dicts = data['shapes']
    points = []
    aug_shape_dicts = []
    i = 0
    
    for shape in shape_dicts:
        
        for pairs in shape['points']:
            points.append(ia.Keypoint(x=pairs[0], y=pairs[1]))
        
        _d = {}
        _d['label'] = shape['label']
        _d['index'] = (i, i+len(shape['points']))
        aug_shape_dicts.append(_d)

        i += len(shape['points'])
    
    keypoints = ia.KeypointsOnImage(points, shape=(256,256,3))
    
    image_aug = seq_det.augment_images([im])[0]
    keypoints_aug = seq_det.augment_keypoints([keypoints])[0]
    
    for shape in aug_shape_dicts:
        start, end = shape['index']
        aug_points = [[keypoint.x, keypoint.y] for keypoint in keypoints_aug.keypoints[start:end]]
        shape['points'] = aug_points
    
    return image_aug, aug_shape_dicts

In [4]:
ia.seed(1)

seq = iaa.Sequential([
#     iaa.Fliplr(0.5)
    iaa.Multiply((1.1, 1.3)),
#     iaa.Affine(
#         rotate=(-90, 90)
#     ),
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 10))
    )
], random_order=True)

NameError: name 'ia' is not defined

In [5]:
seq_det = seq.to_deterministic()
im_path = os.path.join('hola', '38.jpg')
ann_path = os.path.join('anola', '38.json')

im_aug, aug_shape_dicts = augment_poly(im_path, ann_path)
aug_mask = create_binary_masks(im_aug, aug_shape_dicts)
plot_pair((im_aug, aug_mask), gray=True)

NameError: name 'seq' is not defined

In [6]:
image_list = sorted(os.listdir('images'), key=lambda x: int(x.split('.')[0]))
annot_list = sorted(os.listdir('annotated'), key=lambda x: int(x.split('.')[0]))

for i, (im_fn, ann_fn) in enumerate(zip(image_list, annot_list)):
    
    seq_det = seq.to_deterministic()
    im_path = os.path.join('images', im_fn)
    ann_path = os.path.join('annotated', ann_fn)
    im_aug, aug_shape_dicts = augment_poly(im_path, ann_path)
    aug_mask = create_binary_masks(im_aug, aug_shape_dicts)
    
    im = cv2.imread(im_path, 1)
    im_color = draw_multi_masks(im, aug_shape_dicts)
    
    plt.imshow(im_aug)
    plt.show()
    plot_pair((aug_mask, im_color), gray=True)
    
    if i == 4:
        break

NameError: name 'os' is not defined